In [1]:
import re
import os
import time
import datetime
import random
import requests
import numpy as np
import pandas as pd
import ws_func
from datetime import datetime
from os import system   
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains

## Scrape CNN Articles

In [ ]:
# Scrape CNN articles and save to cnn.tsv (tab separated to avoid comma issues with text body)
# ws_func.scrape_cnn(start=(2019, 1, 1), stop=(2020, 1, 1), ret_csv=True, csv='cnn.tsv', ret_df=False)

### Notes on CNN Articles
- Total number of articles: 221
- Date range of scrape: 2019 Jan 1 to 2019 Dec 31 (future scrapes from 2019 Mar 15 to 2019 Dec 31)
- Dropped (within the script) all 'live coverage' and 'style' section articles
- Articles with text: 220
- Articles w/o text: 1 (index: 205)
- Article without text is summary article (to be dropped)

In [ ]:
df = pd.read_csv('cnn.tsv', sep='\t', index_col=0)

In [2]:
urls, dates = ws_func.scrape_urls_scmp(start=(2019, 5, 16), stop=(2020, 3, 29), scroll=100)

Scrolls: 1
Scrolls: 2
Scrolls: 3
Scrolls: 4
Scrolls: 5
Scrolls: 6


KeyboardInterrupt: 

In [ ]:
random.randint(1, 3)

In [ ]:
driver = webdriver.Chrome()
driver.get('https://www.scmp.com/topics/hong-kong-protests')

In [ ]:
def super_scroll(scroll):
    i = 0
    while i < scroll:
        actions = ActionChains(driver)
        time.sleep(2)
        more_content = driver.find_element_by_class_name('topic-content__load-more-anchor')
        actions.move_to_element(more_content).double_click(more_content).send_keys(Keys.SPACE).perform()
        i += 1
        print(f'Scrolls: {i}')

In [ ]:
super_scroll(scroll=2)

In [ ]:
def len_eles():
    len_eles = len(driver.find_elements_by_xpath('//*[@id="topic-detail"]/div[1]/div/div[5]/div[2]/*'))
    return len_eles

In [ ]:
xpath = '//*[@id="topic-detail"]/div[1]/div/div[5]/div[2]/div[{}]/div[1]/div[2]/div[1]/div/span'
a_xpath = '//*[@id="topic-detail"]/div[1]/div/div[5]/div[2]/div[{}]/div[1]/div[1]/div/div/div[2]/a'
dates = []
urls = []
start = datetime(2019, 3, 15).date()
stop = datetime(2020, 3, 30).date()

In [ ]:
start_count = 0
stop_count = len_eles()

In [ ]:
def scrape_urls(urls, dates, start, stop, start_count, stop_count):
    counter = 0
    for i in range(start_count, stop_count):
        try:
            ele = driver.find_element_by_xpath(xpath.format(i+1)).text[:11]
            ts = datetime.strptime(ele, '%d %b %Y').date()
            href = driver.find_element_by_xpath(a_xpath.format(i+1)).get_attribute('href')
            if ts < start:
                counter += 1
                if counter > 5:
                    print('All articles after start date scraped')
                    return urls, date
                continue
            elif ts < stop:
                if re.search('(news)', href):
                    urls.append(href)
                    dates.append(ts)    
        except:
            continue
    return urls, dates

In [ ]:
urls, dates = scrape_urls(urls, dates, start, stop, start_count, stop_count)

In [ ]:
urls

In [ ]:
driver.get(urls[0])
driver.find_element_by_class_name('info__headline').text

In [ ]:
modal_button = driver.find_element_by_css_selector('svg')
modal_button.click()

In [ ]:
driver.find_element_by_class_name('main-info__names').text

In [ ]:
driver.find_element_by_class_name('button-container__button').click()

In [ ]:
texts = driver.find_elements_by_class_name('generic-article__body')
article = ' '.join([ text.text for text in texts ])
byline = driver.find_element_by_class_name('main-info__names').text

In [ ]:
article

In [ ]:
def url_scraper(urls, dates, opin, start, stop):
    actions = ActionChains(driver)
    more_content = driver.find_element_by_class_name('topic-content__load-more-anchor')
    more_move = actions.move_to_element(more_content).perform()
    space = actions.send_keys(Keys.SPACE).perform()
    
    start_count = 0
    stop_count = len_eles()
    

In [ ]:
urls = []
dates = []
opin = []
url_scraper(urls, dates, opin)

In [ ]:
elements = driver.find_elements_by_xpath(xpath.format(i+1))
len(elements)

In [ ]:
//*[@id="topic-detail"]/div[1]/div/div[5]/div[2]/div

In [ ]:
elements = driver.find_elements_by_xpath('//*[@id="topic-detail"]/div[1]/div/div[5]/div[2]/*')

In [ ]:
len(elements)

In [ ]:
driver.quit()